In [1]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)


In [3]:
model = tensorflow.keras.models.load_model('keras_model.h5')

In [4]:
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open('test_nomask.jpg')

#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)
#turn the image into a numpy array
image_array = np.asarray(image)

In [5]:
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open('test_nomask.jpg')

#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)
#turn the image into a numpy array
image_array = np.asarray(image)

# display the resized image
image.show()

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

# Load the image into the array
data[0] = normalized_image_array

# run the inference
prediction = model.predict(data)
print(prediction)

[[0.00214836 0.9978516 ]]


In [8]:
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
size = (224, 224)

font = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,500)
fontScale = 1
fontColor = (255,255,255)
lineType = 2

def main():
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Format the image into a PIL Image so its compatable with Edge TPU
        cv2_im = frame
        pil_im = Image.fromarray(cv2_im)

        # Resize and flip image so its a square and matches training
        image = ImageOps.fit(pil_im, size, Image.ANTIALIAS)
        image_array = np.asarray(image)
        normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
        data[0] = normalized_image_array
        # Classify and display image
        prediction = model.predict(data)
        cv2.putText(img,prediction, 
            bottomLeftCornerOfText, 
            font, 
            fontScale,
            fontColor,
            lineType)
        cv2.imshow('frame', cv2_im)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [9]:
main()

[[0.2300567  0.76994336]]
[[0.61330646 0.38669357]]
[[0.6119297  0.38807032]]
[[0.12046926 0.8795307 ]]
[[0.5443058  0.45569417]]
[[0.38469505 0.6153049 ]]
[[0.37220523 0.6277947 ]]
[[0.46681848 0.53318155]]
[[0.3983925 0.6016075]]
[[0.36427513 0.6357249 ]]
[[0.46075806 0.53924197]]
[[0.42233643 0.57766354]]
[[0.01039701 0.989603  ]]
[[0.00501495 0.9949851 ]]
[[0.00883977 0.9911602 ]]
[[0.02304903 0.97695094]]
[[0.03661093 0.96338904]]
[[0.19858027 0.80141973]]
[[0.03258249 0.96741754]]
[[0.00259038 0.99740964]]
[[0.0629347  0.93706536]]
[[0.02752547 0.9724745 ]]
[[0.01494226 0.9850578 ]]
[[0.00446441 0.9955356 ]]
[[0.00181125 0.9981888 ]]
[[0.00509496 0.994905  ]]
[[0.01240516 0.98759484]]
[[0.00806021 0.99193984]]
[[0.01810925 0.98189074]]
[[0.06299918 0.93700075]]
[[0.01038998 0.98960996]]
[[0.00818448 0.9918155 ]]
[[0.21701218 0.78298783]]
[[0.9325966  0.06740333]]
[[0.99910825 0.00089174]]
[[0.99964786 0.00035216]]
[[0.99983025 0.00016975]]
[[0.99999666 0.00000336]]
[[0.9999795  0